<a href="https://colab.research.google.com/github/GurkiratSarna/Information-Retrieval/blob/master/Assigment%201%20-%20David.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Indexing

### Importing the packages and libraries

In [0]:
pip install spacy

In [0]:
pip install inflect

In [0]:
import nltk, re
import sklearn
import urllib.request
import urllib.parse
import numpy as np
import pandas as pd
import collections
import sys
import urllib
from bs4 import BeautifulSoup


In [0]:
#Installing the package
nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

### HTML Parsing

In [0]:
#reading Web pages
#Web page 1
url1 = "http://www.multimediaeval.org/mediaeval2019/memorability/"
html1= urllib.request.urlopen(url1).read().decode('utf8')
#Web page 2
url2 = "https://sites.google.com/view/siirh2020/"
html2= urllib.request.urlopen(url2).read().decode('utf8')

In [0]:
#Getting the plain text
#Parsing the web pages

#Web page 1
soup1 = BeautifulSoup(html1)

# Delete script and style
for script in soup1(["script", "style"]):
    script.extract()

# get text
text1 = soup1.get_text()

# Remove leading and trailing space in each line
lines1 = (line.strip() for line in text1.splitlines())

# break multi-headlines into each line
chunks1 = (phrase.strip() for line in lines1 for phrase in line.split("  "))

# remove blank lines
text1 = '\n'.join(chunk for chunk in chunks1 if chunk)

#print(text1)

#---------

#Web page 2
soup2 = BeautifulSoup(html2)

# Delete script and style
for script in soup2(["script", "style"]):
    script.extract()

# get text
text2 = soup2.get_text()

# Remove leading and trailing space in each line
lines2 = (line.strip() for line in text2.splitlines())

# break multi-headlines into each line
chunks2 = (phrase.strip() for line in lines2 for phrase in line.split("  "))

# remove blank lines
text2 = '\n'.join(chunk for chunk in chunks2 if chunk)

#print(text2)

### Pre-processing: Sentence Splitting, Tokenization and Normalization

In [0]:
#Setting Splitting
from nltk import sent_tokenize
#Web page 1
split1 = sent_tokenize(text1)
#Web page 2
split2 = sent_tokenize(text2)

In [0]:
#Tokenization
from nltk import word_tokenize

#Web page 1
list_token1=[]
for i in split1:
    token1 = word_tokenize(i)
    list_token1.append(token1)
    
#print(list_token1)

#Web page 2
list_token2=[]
for i in split2:
    token2 = word_tokenize(i)
    list_token2.append(token2)
    
#print(list_token2)

In [0]:
#Normalization

#Removing non-ASCII characters

import inflect

def rep_numbers(token):
    p = inflect.engine()
    words = []
    for i in token:
        if i.isdigit():
            new_word = p.number_to_words(i)
            words.append(new_word)
        else:
            words.append(i)
    return words


#Change words to lowercase
def lowerc(token):
    words = []
    for i in token:
        nrm_word = i.lower()
        words.append(nrm_word)
    return words

#Remove punctuation
def punct(token):
    words = []
    for i in token:
        nrm_word = re.sub(r'[^\w\s]','', i)
        if nrm_word != '':
            words.append(nrm_word)
    return words

#Put everything together
def normalize(token):
    token = rep_numbers(token)
    token = lowerc(token)
    token = punct(token)
    return token

In [0]:
#Calling the normalization function
#Web page 1
norm_tokens1 = []
for i in list_token1:
    norm_token1 = normalize(i)
    norm_tokens1.append(norm_token1)
    
#Web page2
norm_tokens2 = []
for j in list_token2:
    norm_token2 = normalize(j)
    norm_tokens2.append(norm_token2)

In [0]:
#Converting list of list into one list
#Web page 1
norm_list1 = []
for i in norm_tokens1:
    for j in i:
        norm_list1.append(j)
#print(norm_list1)

#Web page 2
norm_list2 = []
for k in norm_tokens2:
    for w in k:
        norm_list2.append(w)
#print(norm_list2)

In [0]:
#Deleting stopwords

import re, string, unicodedata
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

#English Stopwords

stop_words = set(stopwords.words('english'))

#Removin stopwords

#Web page 1
filtered1 = []
for i in norm_list1:
    if i not in stop_words:
        filtered1.append(i)

#print(filtered1)

#Web page 2
filtered2 = []
for j in norm_list2:
    if j not in stop_words:
        filtered2.append(j)

#print(filtered2)


### Lemmatimization and Stemming

In [0]:
#Lemmatimization
from nltk.stem import LancasterStemmer, WordNetLemmatizer

def lemmatimization(filtered):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for i in filtered:
        lemma = lemmatizer.lemmatize(i, pos='v')
        lemmas.append(lemma)
    return lemmas
#Web page 1
lemmas1 = lemmatimization(filtered1)
#print(lemmas1)

#Web page 2
lemmas2 = lemmatimization(filtered2)
#print(lemmas2)

In [0]:
#Stemming

def stemming(lemmas):
    stemmer = LancasterStemmer()
    stems = []
    for i in lemmas:
        stem = stemmer.stem(i)
        stems.append(stem)
    return stems

#Web page 1
stems1 = stemming(lemmas1)
#print(stems1)

#Web page 2
stems2 = stemming(lemmas2)
#print(stems2)

### Part-of-Speech Tagging

In [0]:
#Part-of-Speech Tagging
from nltk import pos_tag

#Web page 1
tag1 = pos_tag(stems1)
#print(tag1)

#Web page 2
tag2 = pos_tag(stems2)
#print(tag2)

## Selecting Keyword

In [16]:
#Counting the frequency of each word for each document
wordSet = set(stems1).union(set(stems2))

wordDictA = dict.fromkeys(wordSet, 0) 
wordDictB = dict.fromkeys(wordSet, 0)

for word in stems1:
    wordDictA[word]+=1
    
for word in stems2:
    wordDictB[word]+=1

pd.DataFrame([wordDictA, wordDictB])

,supercomput,sous,memorabilitysc,cohendet,lasig,read,2014mediaeval,investig,foot,thu,respons,thanhto,rec,italyalfonso,interest,swedenhyeju,greeceg,inform,compos,inf,patternstask,romaniabogd,stateoftheart,two thousand and seventeen,changesport,compil,stockholm,attend,mat,hammad,must,aud,two thousand and twenty,alberto,spainanastasio,francem,rapid,negoty,docu,comp,...,soph,romain,short,techn,ent,object,fut,fil,bioasq,larg,conceiv,changyu,portugalrafael,gwenaël,twentysev,pract,ked,auth,on,pubm,kim,trento,diff,constantin,grow,røst,parissaclay,in,eg,barro,org,issu,may,seventytwo,park,usayonghu,norweg,cho,backgroundth,thirtysix
0,0,0,1,3,0,1,1,0,1,1,1,1,1,0,3,0,0,1,1,1,1,1,1,1,1,0,0,0,1,1,0,2,1,0,0,1,0,0,0,1,...,1,2,1,1,0,2,0,0,0,1,0,1,0,1,0,0,1,0,2,0,0,0,3,1,1,0,0,1,1,0,0,0,2,1,0,0,0,1,1,1
1,2,1,0,0,4,0,0,2,0,0,0,0,0,1,2,1,1,4,0,0,0,0,0,0,0,1,2,1,0,0,1,0,4,1,1,0,1,1,3,0,...,0,0,1,0,1,0,1,1,1,0,1,0,1,0,1,1,0,2,2,1,1,2,2,0,1,1,2,0,0,1,2,2,0,0,1,1,1,0,0,0


In [0]:
#Creating the function to calculate the TF
def computeTF (wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word]=count/float(bowCount)
    return tfDict

In [0]:
#Compute the function for the two documents
tfBowA = computeTF(wordDictA, stems1)
tfBowB = computeTF(wordDictB, stems2)

In [0]:
#Create the fucntion to calculate the IDF
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [0]:
#Computing the IDF for both documents
idfs = computeIDF([wordDictA, wordDictB])

In [0]:
#Create the function to calculate the TF.IDF score
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [0]:
#Calculating the TF.IDF score for both documents
tfidfBowA = computeTFIDF(tfBowA, idfs)
tfidfBowB = computeTFIDF(tfBowB, idfs)

In [24]:
#Data frame of the scores
score = pd.DataFrame([tfidfBowA, tfidfBowB])
score

,supercomput,sous,memorabilitysc,cohendet,lasig,read,2014mediaeval,investig,foot,thu,respons,thanhto,rec,italyalfonso,interest,swedenhyeju,greeceg,inform,compos,inf,patternstask,romaniabogd,stateoftheart,two thousand and seventeen,changesport,compil,stockholm,attend,mat,hammad,must,aud,two thousand and twenty,alberto,spainanastasio,francem,rapid,negoty,docu,comp,...,soph,romain,short,techn,ent,object,fut,fil,bioasq,larg,conceiv,changyu,portugalrafael,gwenaël,twentysev,pract,ked,auth,on,pubm,kim,trento,diff,constantin,grow,røst,parissaclay,in,eg,barro,org,issu,may,seventytwo,park,usayonghu,norweg,cho,backgroundth,thirtysix
0,0.000000,0.000000,0.00043,0.00129,0.000000,0.00043,0.00043,0.000000,0.00043,0.00043,0.00043,0.00043,0.00043,0.000000,0.0,0.000000,0.000000,0.0,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.00043,0.000000,0.000000,0.000000,0.00043,0.00043,0.000000,0.00086,0.0,0.000000,0.000000,0.00043,0.000000,0.000000,0.000000,0.00043,...,0.00043,0.00086,0.0,0.00043,0.000000,0.00086,0.000000,0.000000,0.000000,0.00043,0.000000,0.00043,0.000000,0.00043,0.000000,0.000000,0.00043,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00043,0.0,0.000000,0.000000,0.00043,0.00043,0.000000,0.000000,0.000000,0.00086,0.00043,0.000000,0.000000,0.000000,0.00043,0.00043,0.00043
1,0.000992,0.000496,0.00000,0.00000,0.001984,0.00000,0.00000,0.000992,0.00000,0.00000,0.00000,0.00000,0.00000,0.000496,0.0,0.000496,0.000496,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000496,0.000992,0.000496,0.00000,0.00000,0.000496,0.00000,0.0,0.000496,0.000496,0.00000,0.000496,0.000496,0.001488,0.00000,...,0.00000,0.00000,0.0,0.00000,0.000496,0.00000,0.000496,0.000496,0.000496,0.00000,0.000496,0.00000,0.000496,0.00000,0.000496,0.000496,0.00000,0.000992,0.0,0.000496,0.000496,0.000992,0.0,0.00000,0.0,0.000496,0.000992,0.00000,0.00000,0.000496,0.000992,0.000992,0.00000,0.00000,0.000496,0.000496,0.000496,0.00000,0.00000,0.00000


In [0]:
#Sort the scores (to know what words are more relevant in each document)

#Document 1 (Web page 1)
score1 = pd.DataFrame([tfidfBowA])
scorest1 = score1.T
scorest1.columns = ['tf.idf(doc1)']

scorestf1 = scorest1.sort_values(['tf.idf(doc1)'], ascending = False)
#print(scorestf1)

#-------

#Document 2 (Web page 2)
score2 = pd.DataFrame([tfidfBowB])
scorest2 = score2.T
scorest2.columns = ['tf.idf(doc2)']

scorestf2 = scorest2.sort_values(['tf.idf(doc2)'], ascending = False)
#print(scorestf2)

In [0]:
#Exportin the results to csv

scorestf1.to_csv('/Users/davidalbuja/Documents/Essex/Spring/Information Retrieval/Assigment 1/Outputs/TFIDF(doc1).csv')
scorestf2.to_csv('/Users/davidalbuja/Documents/Essex/Spring/Information Retrieval/Assigment 1/Outputs/TFIDF(doc2).csv') 

### Entity recognition

In [0]:
#Entity recognition
from nltk import ne_chunk

chu1 = ne_chunk(tag1, binary=True)
chu2 = ne_chunk(tag2, binary=True)